# **Advanced Hospital CRM with Neo4j**

This notebook demonstrates how to build an advanced Hospital Customer Relationship Management (CRM) system using Neo4j, a graph database optimized for managing complex relationships between data entities.

## **Table of Contents**

1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
3. [Database Schema Design](#Database-Schema-Design)
4. [Connecting to Neo4j](#Connecting-to-Neo4j)
5. [Creating Nodes and Relationships](#Creating-Nodes-and-Relationships)
6. [Populating the Database](#Populating-the-Database)
7. [Advanced Queries](#Advanced-Queries)
    - a. [Find all appointments for a patient](#a-Find-all-appointments-for-a-patient)
    - b. [Find all patients scheduled with a specific doctor](#b-Find-all-patients-scheduled-with-a-specific-doctor)
8. [Implementing Treatment and Medication Nodes](#Implementing-Treatment-and-Medication-Nodes)
9. [Advanced Features](#Advanced-Features)
    - a. [Appointment Reminders](#a-Appointment-Reminders)
    - b. [Analyzing Doctor Workloads](#b-Analyzing-Doctor-Workloads)
10. [Closing the Connection](#Closing-the-Connection)
11. [Conclusion](#Conclusion)

## **Introduction**

The hospital CRM system will manage patients, doctors, appointments, treatments, and other related entities. Neo4j allows efficient modeling and querying of complex relationships such as:

- **Patients** scheduled for **Appointments** with **Doctors**
- **Doctors** specialized in certain **Departments**
- **Patients** undergoing **Treatments** and prescribed **Medications**

## **Prerequisites**

- **Neo4j Database**: Ensure you have Neo4j installed and running.
- **Python 3.x**: For scripting and interacting with the database.
- **Neo4j Python Driver**: Install using `pip install neo4j`

## **Database Schema Design**

### **Nodes**

- **Patient**: Represents a patient.
- **Doctor**: Represents a doctor.
- **Appointment**: Represents an appointment.
- **Department**: Represents a medical department.
- **Treatment**: Represents a treatment plan.
- **Medication**: Represents medication prescribed.

### **Relationships**

- `(Patient)-[:HAS_APPOINTMENT]->(Appointment)`
- `(Appointment)-[:WITH_DOCTOR]->(Doctor)`
- `(Doctor)-[:WORKS_IN]->(Department)`
- `(Patient)-[:UNDERGOES]->(Treatment)`
- `(Treatment)-[:INCLUDES_MEDICATION]->(Medication)`

## **Connecting to Neo4j**

First, establish a connection to the Neo4j database.

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
Neo4j_pass = os.environ['NEO4J_PASS']

In [3]:
from neo4j import GraphDatabase

# Replace with your Neo4j credentials
uri = "bolt://localhost:7687"
username = "neo4j"
password = Neo4j_pass

driver = GraphDatabase.driver(uri, auth=(username, password))

## **Creating Nodes and Relationships**

Define functions to create nodes and relationships.

In [4]:
def create_patient(tx, patient_id, name, age, contact):
    tx.run(
        """
        MERGE (p:Patient {patient_id: $patient_id})
        SET p.name = $name, p.age = $age, p.contact = $contact
        """,
        patient_id=patient_id, name=name, age=age, contact=contact
    )

def create_doctor(tx, doctor_id, name, specialty):
    tx.run(
        """
        MERGE (d:Doctor {doctor_id: $doctor_id})
        SET d.name = $name, d.specialty = $specialty
        """,
        doctor_id=doctor_id, name=name, specialty=specialty
    )

def create_appointment(tx, appointment_id, date, time):
    tx.run(
        """
        MERGE (a:Appointment {appointment_id: $appointment_id})
        SET a.date = $date, a.time = $time
        """,
        appointment_id=appointment_id, date=date, time=time
    )

def link_patient_appointment(tx, patient_id, appointment_id):
    tx.run(
        """
        MATCH (p:Patient {patient_id: $patient_id}), (a:Appointment {appointment_id: $appointment_id})
        MERGE (p)-[:HAS_APPOINTMENT]->(a)
        """,
        patient_id=patient_id, appointment_id=appointment_id
    )

def link_appointment_doctor(tx, appointment_id, doctor_id):
    tx.run(
        """
        MATCH (a:Appointment {appointment_id: $appointment_id}), (d:Doctor {doctor_id: $doctor_id})
        MERGE (a)-[:WITH_DOCTOR]->(d)
        """,
        appointment_id=appointment_id, doctor_id=doctor_id
    )

## **Populating the Database**

Use the above functions to add data.

In [5]:
with driver.session() as session:
    # Create Patients
    session.write_transaction(create_patient, "P001", "Alice Smith", 30, "555-1234")
    session.write_transaction(create_patient, "P002", "Bob Johnson", 45, "555-5678")
    
    # Create Doctors
    session.write_transaction(create_doctor, "D001", "Dr. Emily Brown", "Cardiology")
    session.write_transaction(create_doctor, "D002", "Dr. John Doe", "Neurology")
    
    # Create Appointments
    session.write_transaction(create_appointment, "A001", "2023-10-15", "10:00")
    session.write_transaction(create_appointment, "A002", "2023-10-16", "14:00")
    
    # Link Patients to Appointments
    session.write_transaction(link_patient_appointment, "P001", "A001")
    session.write_transaction(link_patient_appointment, "P002", "A002")
    
    # Link Appointments to Doctors
    session.write_transaction(link_appointment_doctor, "A001", "D001")
    session.write_transaction(link_appointment_doctor, "A002", "D002")

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/97919563.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_patient, "P001", "Alice Smith", 30, "555-1234")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/97919563.py:4: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_patient, "P002", "Bob Johnson", 45, "555-5678")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/97919563.py:7: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_doctor, "D001", "Dr. Emily Brown", "Cardiology")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/97919563.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_doctor, "D002", "Dr. John Doe", "Neurology")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_587

## **Advanced Queries**

### **a. Find all appointments for a patient**

In [6]:
def get_patient_appointments(tx, patient_id):
    result = tx.run(
        """
        MATCH (p:Patient {patient_id: $patient_id})-[:HAS_APPOINTMENT]->(a)-[:WITH_DOCTOR]->(d)
        RETURN a.appointment_id AS appointment_id, a.date AS date, a.time AS time, d.name AS doctor
        """,
        patient_id=patient_id
    )
    return result.data()

with driver.session() as session:
    appointments = session.read_transaction(get_patient_appointments, "P001")
    for record in appointments:
        print(record)

{'appointment_id': 'A001', 'date': '2023-10-15', 'time': '10:00', 'doctor': 'Dr. Emily Brown'}


/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/285344441.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  appointments = session.read_transaction(get_patient_appointments, "P001")


**Output:**

```
{'appointment_id': 'A001', 'date': '2023-10-15', 'time': '10:00', 'doctor': 'Dr. Emily Brown'}
```

### **b. Find all patients scheduled with a specific doctor**

In [7]:
def get_doctor_patients(tx, doctor_id):
    result = tx.run(
        """
        MATCH (d:Doctor {doctor_id: $doctor_id})<-[:WITH_DOCTOR]-(a)<-[:HAS_APPOINTMENT]-(p)
        RETURN p.patient_id AS patient_id, p.name AS name
        """,
        doctor_id=doctor_id
    )
    return result.data()

with driver.session() as session:
    patients = session.read_transaction(get_doctor_patients, "D001")
    for record in patients:
        print(record)

{'patient_id': 'P001', 'name': 'Alice Smith'}


/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/163540448.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  patients = session.read_transaction(get_doctor_patients, "D001")


**Output:**

```
{'patient_id': 'P001', 'name': 'Alice Smith'}
```

## **Implementing Treatment and Medication Nodes**

In [8]:
def create_treatment(tx, treatment_id, description):
    tx.run(
        """
        MERGE (t:Treatment {treatment_id: $treatment_id})
        SET t.description = $description
        """,
        treatment_id=treatment_id, description=description
    )

def create_medication(tx, medication_id, name, dosage):
    tx.run(
        """
        MERGE (m:Medication {medication_id: $medication_id})
        SET m.name = $name, m.dosage = $dosage
        """,
        medication_id=medication_id, name=name, dosage=dosage
    )

def link_treatment_medication(tx, treatment_id, medication_id):
    tx.run(
        """
        MATCH (t:Treatment {treatment_id: $treatment_id}), (m:Medication {medication_id: $medication_id})
        MERGE (t)-[:INCLUDES_MEDICATION]->(m)
        """,
        treatment_id=treatment_id, medication_id=medication_id
    )

def link_patient_treatment(tx, patient_id, treatment_id):
    tx.run(
        """
        MATCH (p:Patient {patient_id: $patient_id}), (t:Treatment {treatment_id: $treatment_id})
        MERGE (p)-[:UNDERGOES]->(t)
        """,
        patient_id=patient_id, treatment_id=treatment_id
    )

**Populate Treatment and Medication Data:**

In [9]:
with driver.session() as session:
    # Create Treatments
    session.write_transaction(create_treatment, "T001", "Heart Disease Treatment")
    
    # Create Medications
    session.write_transaction(create_medication, "M001", "Aspirin", "100mg daily")
    session.write_transaction(create_medication, "M002", "Beta Blockers", "50mg daily")
    
    # Link Treatments to Medications
    session.write_transaction(link_treatment_medication, "T001", "M001")
    session.write_transaction(link_treatment_medication, "T001", "M002")
    
    # Link Patients to Treatments
    session.write_transaction(link_patient_treatment, "P001", "T001")

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/999400858.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_treatment, "T001", "Heart Disease Treatment")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/999400858.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_medication, "M001", "Aspirin", "100mg daily")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/999400858.py:7: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_medication, "M002", "Beta Blockers", "50mg daily")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/999400858.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(link_treatment_medication, "T001", "M001")
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/999

## **Advanced Features**

### **a. Appointment Reminders**

Add properties to appointments for reminders.

In [10]:
def set_appointment_reminder(tx, appointment_id, reminder_sent):
    tx.run(
        """
        MATCH (a:Appointment {appointment_id: $appointment_id})
        SET a.reminder_sent = $reminder_sent
        """,
        appointment_id=appointment_id, reminder_sent=reminder_sent
    )

with driver.session() as session:
    session.write_transaction(set_appointment_reminder, "A001", False)

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/650670045.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(set_appointment_reminder, "A001", False)


### **b. Analyzing Doctor Workloads**

In [11]:
def get_doctor_workload(tx):
    result = tx.run(
        """
        MATCH (d:Doctor)<-[:WITH_DOCTOR]-(a)
        RETURN d.name AS doctor, COUNT(a) AS appointments
        ORDER BY appointments DESC
        """
    )
    return result.data()

with driver.session() as session:
    workloads = session.read_transaction(get_doctor_workload)
    for record in workloads:
        print(record)

{'doctor': 'Dr. Emily Brown', 'appointments': 1}
{'doctor': 'Dr. John Doe', 'appointments': 1}


/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_58705/3288463594.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  workloads = session.read_transaction(get_doctor_workload)


**Output:**

```
{'doctor': 'Dr. Emily Brown', 'appointments': 1}
{'doctor': 'Dr. John Doe', 'appointments': 1}
```

## **Closing the Connection**

Always close the driver connection when done.

In [12]:
driver.close()

## **Conclusion**

This notebook provides a foundation for an advanced hospital CRM system using Neo4j. It demonstrates how to model complex relationships inherent in hospital data and how to interact with the database using Python. You can extend this system by adding more features, optimizing queries, and integrating with other services.

**Enhancements to Consider:**

- **User Authentication and Authorization**: Implement role-based access control.
- **API Layer**: Use a web framework like Flask or Django to create RESTful APIs.
- **Scheduling Algorithms**: Optimize appointment scheduling based on doctor availability.
- **Data Analysis**: Use Neo4j's graph algorithms library for advanced analytics.
- **Notification System**: Integrate with email/SMS services for reminders and notifications.
- **Front-end Interface**: Build an interactive UI using frameworks like React or Angular.

---

**Note**: Ensure that you handle sensitive data securely, complying with regulations like HIPAA if this system is used in a real-world healthcare setting.